<a href="https://colab.research.google.com/github/ranianasser/ass.4/blob/main/MK_code_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.applications.mobilenet import MobileNet
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [7]:
batch_size = 32
img_size=224

train_generator=ImageDataGenerator(
    rescale=1./255
).flow_from_directory(
        '/content/drive/MyDrive/MK/train',
target_size = (img_size, img_size),batch_size = batch_size)

test_generator=ImageDataGenerator(
    rescale=1./255
).flow_from_directory(
    '/content/drive/MyDrive/MK/test',
target_size = (img_size, img_size),batch_size = batch_size)

Found 104 images belonging to 2 classes.
Found 24 images belonging to 2 classes.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
imgs,labels=next(train_generator)

In [9]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(imgs, labels, test_size=0.15, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(27, 224, 224, 3)
(27, 2)
(5, 224, 224, 3)
(5, 2)


In [10]:
from keras.applications.mobilenet import MobileNet
mobilenet=MobileNet()
model=Sequential()
for layer in mobilenet.layers[:-1]:
  model.add(layer)

for layer in model.layers:
  layer.trainable = True # Not trainable weights
model.add(Dense(2, activation='softmax'))
model.summary()

17225924/17225924 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128       
 zation)                                                 

In [11]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [12]:
# Create an ImageDataGenerator object
data_augmentation = ImageDataGenerator(
    rotation_range=30,  #randomly rotated by up to 20 degrees in either direction.
    width_shift_range=0.3,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.3,  # randomly shift images vertically (fraction of total height)
    zoom_range=0.2,  # set range for random zoom
    horizontal_flip=True,  # randomly flip images
)

train_data_augmentation = data_augmentation.flow(X_train, y_train)
validation_data_augmentation = data_augmentation.flow(X_val, y_val)

In [13]:
# Fit Model
early_stopping_monitor = EarlyStopping(patience = 2)
model.fit(train_data_augmentation, validation_data=validation_data_augmentation, epochs = 20, callbacks = [early_stopping_monitor])

Epoch 1/20
1/1 [==============================] - 17s 17s/step - loss: 2.4364 - accuracy: 0.5926 - val_loss: 0.0042 - val_accuracy: 1.0000
Epoch 2/20
1/1 [==============================] - 8s 8s/step - loss: 1.5735 - accuracy: 0.7407 - val_loss: 2.1977 - val_accuracy: 0.8000
Epoch 3/20
1/1 [==============================] - 7s 7s/step - loss: 3.1611e-06 - accuracy: 1.0000 - val_loss: 4.0644 - val_accuracy: 0.4000


In [14]:
test_labels=test_generator.classes
predictions=np.argmax(model.predict(test_generator),axis=1)
print(predictions[:5])

1/1 [==============================] - 3s 3s/step
[0 0 1 1 1]


In [15]:
confusion_matrix(test_labels,predictions)

array([[6, 6],
       [8, 4]])

In [16]:
# Evaluate the model
accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.4166666666666667
Precision: 0.4
Recall: 0.3333333333333333
F1-score: 0.3636363636363636
